In [1]:
import os
import pandas as pd
import geopandas as gp
pd.set_option('display.max_columns', None)

#Step 0: Load in ward shapefile and ER as gdf/df
oct22 = gp.read_file("../raw-from-source/WI_Statewide_WardsOct14_2022/Statewide_WardsOct14_2022.shp")


#sample ER
atg = pd.read_excel("../raw-from-source/election_data/Ward by Ward Report_Attorney General_0.xlsx",sheet_name=1, header=10)
atg = atg.fillna(method='ffill', axis=0)
atg.rename( columns={'Unnamed: 0':'county', 'Unnamed: 1':'RU', 'Unnamed: 2':'tot_votes'}, inplace=True )
atg = atg[atg["RU"]!="County Totals:"]


## Step 1: GDF Join Preparation

In [2]:
#Create common RU identifier and test join ER and Ward shp


#Step 1: Create RU ID in ward gdf and RU ER
atg["county"] = atg["county"].str.rstrip(" ")
atg["ward_range"] = atg["RU"].str.split(" ").str[-1]
atg["mcd_str"] = atg["RU"].str.split(" ").str[0:-2].apply(lambda x: " ".join(x)).str.upper()
atg["county_mcd"] = atg["county"]+";"+atg["mcd_str"]
atg["county_mcd_ward"] = atg["county"]+";"+atg["mcd_str"]+";"+atg["ward_range"]

oct22["CTV"] = oct22["LABEL"].str.split(" ").str[-2]
oct22["MCD_NAME"] = oct22["LABEL"].str.split(" ").str[0:-3].apply(lambda x: " ".join(x)).str.upper()
wards22=oct22.copy()
wards22["county_mcd_ward"] = wards22["CNTY_NAME"].str.upper()+";"+wards22["CTV"].apply(lambda x: {"C":"CITY OF ", "V":"VILLAGE OF ", "T":"TOWN OF "}.get(x))+wards22["MCD_NAME"].str.upper()+";"+wards22["WARD_FIPS"].str.slice(start=-3)


#Step 2: From ER RUs create ward:RU dictionary (Alternatively do this from shp? Using ER as this is the goal set of RUs as oppsed to SHP with IDs from different time period - also ward range not clear in ward shp, that is needed from ER)
ward_dict={}
for ru in atg["county_mcd_ward"]:
    county_str = atg.loc[atg["county_mcd_ward"]==ru, "county"].to_string(index=False)
    mcd_str =  atg.loc[atg["county_mcd_ward"]==ru, "mcd_str"].to_string(index=False).upper()
    #print("RU: ",ru)
    input_wards = ru.split(";")[-1]
    
    if "," in ru.split(";")[-1] and "-" not in ru.split(";")[-1]:
        for x in list(input_wards.split(",")):
            ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
            ward_dict[ward_id_str] = ru
            #print(ward_id_str, ":", ru)
    elif '1-2A' in ru.split(";")[-1]:
        ward_dict["WINNEBAGO;TOWN OF WOLF RIVER;001"] = ru
        ward_dict["WINNEBAGO;TOWN OF WOLF RIVER;002"] = ru
    elif "-" in ru.split(";")[-1] and "," not in ru.split(";")[-1]:
        i = input_wards.split("-")[0]
        j = input_wards.split("-")[1]
        for x in range(int(i), int(j)+1):
            ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
            ward_dict[ward_id_str] = ru
            #print(ward_id_str, ":", ru)
    elif "," in ru.split(";")[-1] and "-" in ru.split(";")[-1]:
        for item in list(input_wards.split(",")):
            #print("item: ", item)
            if "-" in item:
                i = item.split("-")[0]
                j = item.split("-")[1]
                #print("i, j:", i, j)
                for x in range(int(i), int(j)+1):
                    ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
                    ward_dict[ward_id_str] = ru
                    #print(ward_id_str, ":", ru)
            else:
                x = item
                ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
                ward_dict[ward_id_str] = ru
                #print(ward_id_str, ":", ru)
    else:
        x = input_wards
        ward_id_str = county_str+";"+mcd_str+";"+str(x).zfill(3)
        ward_dict[ward_id_str] = ru
        #print(ward_id_str, ":", ru)

        
ward_dict['BAYFIELD;TOWN OF GRANDVIEW;001']= 'BAYFIELD;TOWN OF GRAND VIEW;1'
ward_dict['CRAWFORD;VILLAGE OF MOUNT STERLING;001'] = 'CRAWFORD;VILLAGE OF MT. STERLING;1'
ward_dict['SAUK;VILLAGE OF LA VALLE;001'] = 'SAUK;VILLAGE OF LAVALLE;1'
ward_dict["VILAS;TOWN OF LAND O'LAKES;001"] = 'VILAS;TOWN OF LAND O-LAKES;1-2'
ward_dict["VILAS;TOWN OF LAND O'LAKES;002"] = 'VILAS;TOWN OF LAND O-LAKES;1-2'
ward_dict['WALWORTH;VILLAGE OF FONTANA-ON-GENEVA LAKE;001'] = 'WALWORTH;VILLAGE OF FONTANA;1-3'
ward_dict['WALWORTH;VILLAGE OF FONTANA-ON-GENEVA LAKE;002'] = 'WALWORTH;VILLAGE OF FONTANA;1-3'
ward_dict['WALWORTH;VILLAGE OF FONTANA-ON-GENEVA LAKE;003'] = 'WALWORTH;VILLAGE OF FONTANA;1-3'
ward_dict['WAUPACA;TOWN OF ST. LAWRENCE;001'] = 'WAUPACA;TOWN OF SAINT LAWRENCE;1-2'
ward_dict['WAUPACA;TOWN OF ST. LAWRENCE;002'] = 'WAUPACA;TOWN OF SAINT LAWRENCE;1-2'
ward_dict['WINNEBAGO;TOWN OF WOLF RIVER;02A'] = 'WINNEBAGO;TOWN OF WOLF RIVER;1-2A'
ward_dict['CHIPPEWA;CITY OF CHIPPEWA FALLS;007'] = 'CHIPPEWA;CITY OF CHIPPEWA FALLS;8-7'
ward_dict['CHIPPEWA;CITY OF CHIPPEWA FALLS;008'] = 'CHIPPEWA;CITY OF CHIPPEWA FALLS;8-7'
#See readme for reference to Kenosha
ward_dict['KENOSHA;CITY OF KENOSHA;065'] = 'KENOSHA;TOWN OF SOMERS;2-3'
#ward_dict['CALUMET;TOWN OF HARRISON;001'] = "CALUMET;CITY OF MENASHA;14-16"
ward_dict['CALUMET;TOWN OF HARRISON;001'] = "CALUMET;TOWN OF HARRISON;001"



#Step 3: Apply ward:RU dictionary to ward gdf
wards22["RU_assignment"] = wards22["county_mcd_ward"].map(ward_dict)
gdf_wards_with_match = wards22[~wards22["RU_assignment"].isna()]


#Step 4: Check if any wards do not have valid RU assignment
gdf_wards_no_match = wards22[wards22["RU_assignment"].isna()]
print("Shape wards no match df: ", gdf_wards_no_match.shape)


#Step 5: Test join dissolved gdf<>df
ru_gdf = gdf_wards_with_match.dissolve(by="RU_assignment").reset_index().to_crs(wards22.crs)
prelim_shp_atger_gdf = ru_gdf.merge(atg, how="outer", left_on="RU_assignment", right_on="county_mcd_ward", indicator=True)
print("Wards not assigned: ",wards22["RU_assignment"].isna().shape,"\nWards with match shape: ", gdf_wards_with_match.shape, "\nFrom what should match, did not join: ", prelim_shp_atger_gdf[prelim_shp_atger_gdf["_merge"]!="both"].shape)


Shape wards no match df:  (0, 16)
Wards not assigned:  (6971,) 
Wards with match shape:  (6971, 16) 
From what should match, did not join:  (2, 27)


In [3]:
set_dict_notin_ogid = set(ward_dict.keys())-set(wards22["county_mcd_ward"])
#^We can explain this set easily because of the manual additions to the dict
set_in_ogid_notin_dict = set(wards22["county_mcd_ward"])-set(ward_dict.keys())

In [4]:
wards22.shape

(6971, 16)

In [5]:
len(ward_dict)

6984

In [6]:
len(set_in_ogid_notin_dict)

0

In [7]:
set_in_ogid_notin_dict

set()

#Note it is ok that the ward dict has more items than IDs we need - this is because more were added to modify assignment where needed so the "bad" assignment is in there and gets overridden

#Ran below code to confirm shapes and locations for mismatches
kenosha65 = oct22[oct22["LABEL"].str.upper().str.contains("KENOSHA - C 0065")]
july22 = gp.read_file("./Wisconsin_Redistricting_Wards_(2022)/Wisconsin_Redistricting_Wards_(2022).shp").to_crs(kenosha65.crs)
july22.to_file("./shp_to_check/july22.shp")
kenosha65.to_file("./shp_to_check/kenosha.shp")
oct22[(oct22["LABEL"].str.upper().str.contains("HARRISON - T"))&oct22["CNTY_NAME"].str.upper().str.contains("CALUMET")].to_file("./shp_to_check/harrison.shp")

## Step 2: Read in Election Results

In [8]:
#Step 0: Set up ER read in with functions
from wi_2022_pber_column_dict import column_dict, script_prod_copy_col_dict


def xlsx_format(df_path, sheet_name=1, header=10):
    df = pd.read_excel(df_path, sheet_name=sheet_name, header=header)
    df = df.fillna(method='ffill', axis=0)
    df.rename( columns={'Unnamed: 0':'county', 'Unnamed: 1':'RU', 'Unnamed: 2':'tot_votes'}, inplace=True )
    df = df[df["RU"]!="County Totals:"]

    df["county"] = df["county"].str.rstrip(" ")
    df["ward_range"] = df["RU"].str.split(" ").str[-1]
    df["mcd_str"] = df["RU"].str.split(" ").str[0:-2].apply(lambda x: " ".join(x)).str.upper()
    df["county_mcd"] = df["county"]+";"+df["mcd_str"]
    df["county_mcd_ward"] = df["county"]+";"+df["mcd_str"]+";"+df["ward_range"]
    return df


#Step 1: Read in non-district election result dfs (atg, sos, tre, gov, uss)
atg = xlsx_format("./Ward by Ward Report_Attorney General_0.xlsx")
atg["G22ATGOWRI"] = atg["SCATTERING"]
sos = xlsx_format("./Ward by Ward Report_Secretary of State_0.xlsx")
sos["G22SOSOWRI"] = sos["SCATTERING"]
tre = xlsx_format("./Ward by Ward Report_State Treasurer_0.xlsx")
tre["G22TREOWRI"] = tre["SCATTERING"]
gov = xlsx_format("./Ward by Ward Report_Governor_0.xlsx")
gov["G22GOVOWRI"] = gov["SCATTERING"]
uss = xlsx_format("./Ward by Ward Report_US Senate_0.xlsx")
uss["G22USSOWRI"] = uss["SCATTERING"]


#Step 2: Join non-district election result dfs together
erdf_1 = atg.merge(sos, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_atg', '_sos'))
erdf_2 = erdf_1.merge(tre, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_erdf1', '_tre'))
erdf_3 = erdf_2.merge(gov, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_erdf2', '_gov'))
erdf = erdf_3.merge(uss, how="outer", on=["county","RU","county_mcd_ward"], suffixes=('_erdf3', '_uss'))


#Step 3: Check if any RUs mismatch across non-district dfs by size 
assert atg.shape[0]==sos.shape[0]==tre.shape[0]==gov.shape[0]==uss.shape[0]==erdf.shape[0]


#Step 4: Translate column names from og to RDH format for non-district ER
erdf.columns = erdf.columns.str.replace(" ", "").str.replace("\n", "")
erdf = erdf.rename(columns=column_dict)


def get_candidate_party_dict(contest, df_path, sheet_name, header):
    df = pd.read_excel(df_path, sheet_name=sheet_name, header=[header-1, header])
    party_list = ["DEM", "REP", "IND", "LIB", "CON"]
    no_match_list = []
    candidate_code_dict = {}
    if contest=="GSU":
        district = sheet_name*2-1
    else:
        district = sheet_name 
    for party in party_list: 
        try:
            #print(party)
            candidate_code_dict[df.xs((party), axis=1).columns[0]] = contest+ str(district).zfill(2) + party[0:1]+ df.xs((party), axis=1).columns[0].split(" ")[-1][:3].upper()

        except:
            #print(party, "hit except")
            no_match_list.append(str(district) + party)
            
    return candidate_code_dict, no_match_list, district


def read_in_df_and_col_dict(contest, path, sheet_count):
    bigdf = pd.DataFrame()
    col_dict = {}
    no_match_list = []
    for i in range(1, sheet_count):
        #print(i)
        if i == 1:
            header=10
        else:
            header=9
        new_return = get_candidate_party_dict(contest, path, i, header)
        #print(new_return)
        new_dict = new_return[0]
        col_dict.update(new_dict)
        no_match_list.append(new_return[1])

        df = xlsx_format(path, i,header)
        df[contest[1:]+"_DIST"] = new_return[2]
        df[contest+"OWRI"] = df["SCATTERING"]
        bigdf = pd.concat([bigdf, df], ignore_index=True)
        #bigdf = bigdf.rename(columns=col_dict)
        
    return bigdf, col_dict, no_match_list


#Step 5: Read in district contests
su = read_in_df_and_col_dict("GSU", "Ward by Ward Report_State Senator_0.xlsx", 18)
sl = read_in_df_and_col_dict("GSL", "./Ward by Ward Report_Representative to the Assembly_0.xlsx", 100)
cong = read_in_df_and_col_dict("GCON", "./Ward by Ward Report_Representative in Congress_0.xlsx", 9)

#Step 5: Combine ER together
erdf_su = pd.merge(erdf, su[0], how="outer", on=["county","RU","county_mcd_ward"], suffixes=("_erdf", "_su"))
erdf_sl = pd.merge(erdf_su, sl[0], how="outer", on=["county","RU","county_mcd_ward"], suffixes=("_erdf_su", "_sl"))
erdf_cong = pd.merge(erdf_sl, cong[0], how="outer", on=["county","RU","county_mcd_ward"], suffixes=("_erdf_sl", "_cong"))
erdf_all = erdf_cong.copy()

#Step 6: Polish columns -- separate .py file itereatively

In [9]:
#Run output and copy paste into .py to edit for read in/application
#su[1]
#cong[1]
#sl[1]
#su[2]
#sl[2]
#cong[2]

## Step 3: Join RU GDF and ER DF together

“UNIQUE_ID”, “COUNTYFP” (3-digitX5-digit), all non-electoral columns from VEST (if not a VEST file, add in a county name and a precinct name as well), all election columns SORTED ALPHABETICALLY (including additional races RDH adds), “geometry”

Remember for readme:
Note: for party, we have D, R, I, C:CON?- Constitution party, LIB:Libertarian?

Note UNIQUE ID in this case is reporting unit, explain in readme RU aggregate of wards
Note COUNTYFP is 3-digit county fips code
Note 'WARD_FIPS', 'CTV', 'MCD_NAME' are identifiers from the ward gdf where ward fips is the unique fips code, CTV notes whether an area is city, town township, etd, MCD_NAME is MCD name

**Note eau claire has no votes, reference email about boundaries 

Note Harrison, kenosha, other weird boundary cases

Note ward boundary file received upon request from WI Elections Commission. This is an unofficial file from October of 2022. This was used as the boundaries correspond to RU's more closely than the official file released in July of 2022 because of changes that occurred between July and October. This is just the timeframe frequency WI releases official files

Note if district column has 0 value means wasnt assigned, weren't votes there, wasn't relevant

Note "The names of persons not regularly nominated who received only a comparatively small number of votes may be omitted and their votes designated as scattering votes." - https://docs.legis.wisconsin.gov/document/statutes/7.75#:~:text=The%20names%20of%20persons%20not,votes%20designated%20as%20scattering%20votes.

In [10]:
#Step 1: Join gdf<>df
from wi_2022_pber_column_dict import script_prod_copy_col_dict
er_gdf_merge = ru_gdf.merge(erdf_all, how="outer", left_on="RU_assignment", right_on="county_mcd_ward", suffixes=("_test_merge", "_ru_gdf"))
er_gdf_merge = er_gdf_merge.rename(columns=script_prod_copy_col_dict)

#Step 2: Filter columns + polish new gdf
election_cols =sorted(list(er_gdf_merge.columns[er_gdf_merge.columns.str.startswith("G")]))
print(election_cols)
er_gdf_merge["UNIQUE_ID"] = er_gdf_merge["RU_assignment"]
er_gdf_merge["COUNTYFP"] = er_gdf_merge["CNTY_FIPS"].str.slice(start=2)
er_gdf_merge["COUNTYNM"] = er_gdf_merge["county"]
er_gdf_merge["SLDL_DIST"] = er_gdf_merge["SL_DIST"]
er_gdf_merge["SLDU_DIST"] = er_gdf_merge["SU_DIST"]


gdf = er_gdf_merge[['UNIQUE_ID','COUNTYFP',"COUNTYNM",'CTV','MCD_NAME',"CON_DIST","SLDL_DIST","SLDU_DIST"] + election_cols + ["geometry"]]

assert (gdf[election_cols][gdf["UNIQUE_ID"].isna()].sum()==0).all()
#See readme for why 'EAU CLAIRE;CITY OF EAU CLAIRE;73', (zero votes) has no geometry and is fine to cut out
gdf = gdf[~gdf["geometry"].isna()].fillna(0)

#Check that no candidate names are not getting assignments
print(list(er_gdf_merge.columns[~er_gdf_merge.columns.str.startswith("G")]))

gdf.loc[gdf["CON_DIST"]==0.0, "CON_DIST"] = "no assignment"
gdf.loc[gdf["SLDU_DIST"]==0.0, "SU_DIST"] = "no assignment"
gdf.loc[gdf["SLDL_DIST"]==0.0, "SL_DIST"] = "no assignment"
gdf.loc[gdf["UNIQUE_ID"]=="CALUMET;TOWN OF HARRISON;001", "COUNTYNM"] = "CALUMET"

['G22ATGDKAU', 'G22ATGOWRI', 'G22ATGRTON', 'G22GOVDEVE', 'G22GOVIBEG', 'G22GOVIHAS', 'G22GOVOWRI', 'G22GOVRMIC', 'G22SOSDFOL', 'G22SOSIMCF', 'G22SOSLHAR', 'G22SOSOWRI', 'G22SOSRLOU', 'G22TRECZUE', 'G22TREDRIC', 'G22TREOWRI', 'G22TRERLEI', 'G22USSDBAR', 'G22USSIPAU', 'G22USSOWRI', 'G22USSRJOH', 'GCON01DROE', 'GCON01IBAR', 'GCON01RSTE', 'GCON02DPOC', 'GCON02IALE', 'GCON02ROLS', 'GCON03DPFA', 'GCON03RORD', 'GCON04DMOO', 'GCON04IRAY', 'GCON04RROG', 'GCON05DSOM', 'GCON05RFIT', 'GCON06IPOW', 'GCON06RGRO', 'GCON07DAUS', 'GCON07RTIF', 'GCON08DHAN', 'GCON08DHOF', 'GCON08IBOU', 'GCON08LVAN', 'GCON08RGAL', 'GCONOWRI', 'GSL01DTHE', 'GSL01RKIT', 'GSL02DGAS', 'GSL02RSOR', 'GSL03RTUS', 'GSL04DTEA', 'GSL04RSTE', 'GSL05DDEU', 'GSL05RGOE', 'GSL06DSWI', 'GSL06RNEU', 'GSL06RSCH', 'GSL07DRIE', 'GSL07RMAR', 'GSL08DORT', 'GSL09DCAB', 'GSL09RANT', 'GSL10DMAD', 'GSL11DDRA', 'GSL12DMYE', 'GSL12RCAN', 'GSL13DHAR', 'GSL13RMIC', 'GSL14DVIN', 'GSL14RTUR', 'GSL15RMAX', 'GSL16DHAY', 'GSL17DOMO', 'GSL17REIS', 'GSL18DG

In [11]:
gdf[gdf["UNIQUE_ID"]=="CALUMET;TOWN OF HARRISON;001"]

,UNIQUE_ID,COUNTYFP,COUNTYNM,CTV,MCD_NAME,CON_DIST,SLDL_DIST,SLDU_DIST,G22ATGDKAU,G22ATGOWRI,G22ATGRTON,G22GOVDEVE,G22GOVIBEG,G22GOVIHAS,G22GOVOWRI,G22GOVRMIC,G22SOSDFOL,G22SOSIMCF,G22SOSLHAR,G22SOSOWRI,G22SOSRLOU,G22TRECZUE,G22TREDRIC,G22TREOWRI,G22TRERLEI,G22USSDBAR,G22USSIPAU,G22USSOWRI,G22USSRJOH,GCON01DROE,GCON01IBAR,GCON01RSTE,GCON02DPOC,GCON02IALE,GCON02ROLS,GCON03DPFA,GCON03RORD,GCON04DMOO,GCON04IRAY,GCON04RROG,GCON05DSOM,GCON05RFIT,GCON06IPOW,GCON06RGRO,GCON07DAUS,GCON07RTIF,GCON08DHAN,GCON08DHOF,GCON08IBOU,GCON08LVAN,GCON08RGAL,GCONOWRI,GSL01DTHE,GSL01RKIT,GSL02DGAS,GSL02RSOR,GSL03RTUS,GSL04DTEA,GSL04RSTE,GSL05DDEU,GSL05RGOE,GSL06DSWI,GSL06RNEU,GSL06RSCH,GSL07DRIE,GSL07RMAR,GSL08DORT,GSL09DCAB,GSL09RANT,GSL10DMAD,GSL11DDRA,GSL12DMYE,GSL12RCAN,GSL13DHAR,GSL13RMIC,GSL14DVIN,GSL14RTUR,GSL15RMAX,GSL16DHAY,GSL17DOMO,GSL17REIS,GSL18DGOY,GSL19DCLA,GSL20DSIN,GSL20RHER,GSL21DJUR,GSL21RROD,GSL22DBRO,GSL22RBRA,GSL23DAND,GSL23RNAT,GSL24DTAT,GSL24RKNO,GSL25RTIT,GSL26DSAL,GSL26RKAT,GSL27IGER,GSL27RBIN,GSL28DSCH,GSL28RMAG,GSL29DJOH,GSL29RMOS,GSL30DYAC,GSL30RZIM,GSL31DBRO,GSL31RSCH,GSL32DJAR,GSL32RAUG,GSL33DVRU,GSL33RJOH,GSL34DDAN,GSL34RSWE,GSL35IFRE,GSL35RCAL,GSL36DMUR,GSL36RMUR,GSL37DMCC,GSL37RPEN,GSL38RDIT,GSL39RBOR,GSL39RRYD,GSL40IFRI,GSL40RPET,GSL41RDAL,GSL42DVAL,GSL42RPLU,GSL43DJAC,GSL43RVOE,GSL44DCON,GSL44RZIM,GSL45DAND,GSL45RKLE,GSL46DRAT,GSL46RMCK,GSL47DAND,GSL47RNEW,GSL48DBAL,GSL49DPAR,GSL49RTRA,GSL50DLEU,GSL50RKUR,GSL51DSPI,GSL51RNOV,GSL52DLAV,GSL52RO'C,GSL53RSCH,GSL54DPAL,GSL54RHER,GSL55DHOL,GSL55RGUS,GSL56DHAY,GSL56RMUR,GSL57DSNO,GSL57RFOX,GSL58DRZE,GSL58RGUN,GSL59RBOD,GSL60DLAR,GSL60RBRO,GSL61DWIN,GSL61RNED,GSL62DHAM,GSL62RWIT,GSL63DJAC,GSL63RSTE,GSL63RVOS,GSL64DMCG,GSL64RHIB,GSL65DOHN,GSL65RPET,GSL66DNEU,GSL66LHUT,GSL67DBEN,GSL67RSUM,GSL68DOTT,GSL68RHUR,GSL69DBOE,GSL69RROZ,GSL70DGOM,GSL70RVAN,GSL71DSHA,GSL71RSOI,GSL72DGRE,GSL72RKRU,GSL73DGAP,GSL73RSAP,GSL74DADA,GSL74RGRE,GSL75RARM,GSL76DHON,GSL77DSTU,GSL78DSUB,GSL78RNEU,GSL79DJOE,GSL79RFUE,GSL80DBAR,GSL80RLUG,GSL81DCON,GSL81RBEN,GSL82DDAV,GSL82IROP,GSL82RWIC,GSL83ISEL,GSL83RRET,GSL84DBIR,GSL84RDON,GSL85DCON,GSL85RSNY,GSL86RSPI,GSL87DRIL,GSL87REDM,GSL88DBEA,GSL88RMAC,GSL89DBEN,GSL89RBEH,GSL90DSHE,GSL90RBEH,GSL91DEME,GSL91RSTA,GSL92DBAM,GSL92RPRO,GSL93DPAG,GSL93RPET,GSL94DDOY,GSL94RHUE,GSL95DBIL,GSL95RWOO,GSL96DSWI,GSL96ROLD,GSL97RALL,GSL98DBAR,GSL98RNEY,GSL99DDAH,GSL99RDUC,GSLOWRI,GSU01DGAG,GSU01RJAC,GSU03DCAR,GSU03RSAN,GSU05DKAT,GSU05RHUT,GSU07DLAR,GSU07RGIL,GSU09DSCH,GSU09RLEM,GSU11DDOE,GSU11RNAS,GSU13RJAG,GSU15DSPR,GSU15RTRO,GSU17DSKO,GSU17RMAR,GSU19DALF,GSU19RCAB,GSU21RWAN,GSU23RJAM,GSU25DWES,GSU25RQUI,GSU27DHES,GSU27RREL,GSU29DLOO,GSU29RTOM,GSU31DSMI,GSU31REST,GSU33RKAP,GSUOWRI,geometry,SU_DIST,SL_DIST
270,CALUMET;TOWN OF HARRISON;001,015,CALUMET,T,HARRISON,no assignment,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-88.39786 44.21431, -88.39786 44.213...",no assignment,no assignment


## Step 4: Check vote totals

In [12]:
def check_unique_id_unique(merged_gdf):
    assert merged_gdf["UNIQUE_ID"].isna().any()==False
    assert merged_gdf["UNIQUE_ID"].nunique()==merged_gdf.shape[0]
    return "unique_id is unique"


def statewide_totals_check(partner_df, partner_name, source_df, source_name, column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
 
    Returns:
      difference list
    """
    print("***Statewide Totals Check***")
    diff_races=[]
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            if race not in diff_races:
                diff_races.append(race)
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\t"+ partner_name + ": "+str(partner_df[race].sum())+" votes")
            print("\t"+ source_name +": "+str(source_df[race].sum())+" votes")
        #else:
            #print(race + " is equal", "\t both dataframes " + str(partner_df[race].sum()))
    
    if (len(diff_races)==0):
        print("All contests match statewide!")
    elif (len(diff_races)>0):
        print("Contests with differences: ")
    
    return diff_races


def county_totals_check(partner_df, partner_name, source_df, source_name, column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      partner_name: String of what to call the partner in the print statement
      source_df: DataFrame of election results we are comparing to
      source_name: String of what to call the source in the print statement
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      difference list
    """
    
    print("\n***Countywide Totals Check***")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race, " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t",val," has a difference of ",str(county_differences[val])," votes")
                print("\t\t", partner_name , ": ",str(partner_df.groupby([county_col]).sum().loc[val,race])," votes")
                print("\t\t", source_name ,": ",str(source_df.groupby([county_col]).sum().loc[val,race])," votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t",val , ": ", str(partner_df.groupby([county_col]).sum().loc[val,race])," votes")
        #else:
            #print(race + " is equal across all counties")
            #if (full_print):
               # for val in diff[diff == 0].index.values.tolist():
                 #   county_similarities = diff[diff == 0]
                    #print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)==0):
        print("All contests in all counties match!")
    elif (len(diff_counties)>0):
        print("Counties with differences: ")
        
    return diff_counties


def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    """Checks a merged dataframe with two election results at the precinct level

    Args:
      merged_df: DataFrame with one set of election results joined to another
      column_list: List of races that there are votes for
      vest_on_left: Boolean specifying whether VEST data is on the left side of merged_df
      name_col: String of the column name to refer to precincts when a difference occurs
      print_level: Integer that specifies how large the vote difference in a precinct must be to be printed

    Returns:
      list of differences
    """
    print("\n***Precinct Totals Check***")
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
            
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1

    print("\nThere are ", len(merged_df.index)," total rows")
    
    if(len(diff_values)!=0):
        print(matching_rows," of these rows are the same")
        print("\nAll precincts containing differences:")
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
        print("\nThe max difference between any one shared column in a row is: ", max_diff)
        count_big_diff = len([i for i in diff_values if i > 10])
        print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    else:
        print(matching_rows," of these rows are the same")
    
    diff_list.sort()
    
    return diff_list


def run_all_checks(partner_df, partner_name, source_df, source_name, column_list,county_col,full_print=False, prec_check=True):
    print(check_unique_id_unique(source_df))
    #Running inner join because of expected nan value for ZZZ precincts
    merged_df = pd.merge(source_df, partner_df, on = ["UNIQUE_ID"], how = "inner", indicator=True)
    vest_on_left = False
    name_col = "UNIQUE_ID"
    #All matches statewide and county levels
    statewide_totals_check(partner_df, partner_name, source_df, source_name, column_list)
    county_totals_check(partner_df, partner_name, source_df, source_name, column_list,county_col,full_print=False)
    if prec_check ==True:
        precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0)
        
import numpy as np

def prep_er_df_for_check_no_dist(erdf):
    erdf["UNIQUE_ID"]=erdf["county_mcd_ward"]        
    erdf["COUNTYNM"] = erdf["county"]    
    erdf.columns = erdf.columns.str.replace(" ", "").str.replace("\n", "")
    erdf = erdf.rename(columns=column_dict)
    
    return erdf


def prep_er_df_for_check_w_dist(erdf):
    erdf["UNIQUE_ID"]=erdf["county_mcd_ward"]        
    erdf["COUNTYNM"] = erdf["county"]    
    #erdf.columns = erdf.columns.str.replace(" ", "").str.replace("\n", "")
    erdf = erdf.rename(columns=script_prod_copy_col_dict)
    erdf = erdf.fillna(0)
    
    return erdf


print("**Running ATG checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_no_dist(atg), "WI", list(gdf.columns[gdf.columns.str.startswith("G22ATG")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running USS checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_no_dist(uss), "WI", list(gdf.columns[gdf.columns.str.startswith("G22USS")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running TRE checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_no_dist(tre), "WI", list(gdf.columns[gdf.columns.str.startswith("G22TRE")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running SOS checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_no_dist(sos), "WI", list(gdf.columns[gdf.columns.str.startswith("G22SOS")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running GOV checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_no_dist(gov), "WI", list(gdf.columns[gdf.columns.str.startswith("G22GOV")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running CONG checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_w_dist(cong[0]), "WI", list(gdf.columns[gdf.columns.str.startswith("GCON")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running SL checks**")
run_all_checks(gdf[["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_w_dist(sl[0]), "WI", list(gdf.columns[gdf.columns.str.startswith("GSL")]),"COUNTYNM",full_print=False, prec_check=True)
print("**Running SU checks**")
#Note subset gdf because SU did not have contests in all counties, so want to just compare those with contests to make fxn work
run_all_checks(gdf[gdf["COUNTYNM"].isin(set(su[0]["county"]))][["UNIQUE_ID","COUNTYNM"]+election_cols], "rdh", prep_er_df_for_check_w_dist(su[0]), "WI", list(gdf.columns[gdf.columns.str.startswith("GSU")]),"COUNTYNM",full_print=False, prec_check=True)

**Running ATG checks**
unique_id is unique
***Statewide Totals Check***
All contests match statewide!

***Countywide Totals Check***
All contests in all counties match!

***Precinct Totals Check***

There are  3626  total rows
3626  of these rows are the same
**Running USS checks**
unique_id is unique
***Statewide Totals Check***
All contests match statewide!

***Countywide Totals Check***
All contests in all counties match!

***Precinct Totals Check***

There are  3626  total rows
3626  of these rows are the same
**Running TRE checks**
unique_id is unique
***Statewide Totals Check***
All contests match statewide!

***Countywide Totals Check***
All contests in all counties match!

***Precinct Totals Check***

There are  3626  total rows
3626  of these rows are the same
**Running SOS checks**
unique_id is unique
***Statewide Totals Check***
All contests match statewide!

***Countywide Totals Check***
All contests in all counties match!

***Precinct Totals Check***

There are  3626  tota

<ipython-input-12-45e98d385bb1>:173: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  erdf["UNIQUE_ID"]=erdf["county_mcd_ward"]
<ipython-input-12-45e98d385bb1>:174: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  erdf["COUNTYNM"] = erdf["county"]


All contests in all counties match!

***Precinct Totals Check***

There are  3626  total rows
3626  of these rows are the same
**Running SU checks**
unique_id is unique
***Statewide Totals Check***
All contests match statewide!

***Countywide Totals Check***
All contests in all counties match!

***Precinct Totals Check***

There are  1925  total rows
1925  of these rows are the same


## Step 5: Export

In [13]:
clean = gdf.copy()
clean["geometry"]=clean.geometry.buffer(0)
clean[~clean["geometry"].is_valid]

,UNIQUE_ID,COUNTYFP,COUNTYNM,CTV,MCD_NAME,CON_DIST,SLDL_DIST,SLDU_DIST,G22ATGDKAU,G22ATGOWRI,G22ATGRTON,G22GOVDEVE,G22GOVIBEG,G22GOVIHAS,G22GOVOWRI,G22GOVRMIC,G22SOSDFOL,G22SOSIMCF,G22SOSLHAR,G22SOSOWRI,G22SOSRLOU,G22TRECZUE,G22TREDRIC,G22TREOWRI,G22TRERLEI,G22USSDBAR,G22USSIPAU,G22USSOWRI,G22USSRJOH,GCON01DROE,GCON01IBAR,GCON01RSTE,GCON02DPOC,GCON02IALE,GCON02ROLS,GCON03DPFA,GCON03RORD,GCON04DMOO,GCON04IRAY,GCON04RROG,GCON05DSOM,GCON05RFIT,GCON06IPOW,GCON06RGRO,GCON07DAUS,GCON07RTIF,GCON08DHAN,GCON08DHOF,GCON08IBOU,GCON08LVAN,GCON08RGAL,GCONOWRI,GSL01DTHE,GSL01RKIT,GSL02DGAS,GSL02RSOR,GSL03RTUS,GSL04DTEA,GSL04RSTE,GSL05DDEU,GSL05RGOE,GSL06DSWI,GSL06RNEU,GSL06RSCH,GSL07DRIE,GSL07RMAR,GSL08DORT,GSL09DCAB,GSL09RANT,GSL10DMAD,GSL11DDRA,GSL12DMYE,GSL12RCAN,GSL13DHAR,GSL13RMIC,GSL14DVIN,GSL14RTUR,GSL15RMAX,GSL16DHAY,GSL17DOMO,GSL17REIS,GSL18DGOY,GSL19DCLA,GSL20DSIN,GSL20RHER,GSL21DJUR,GSL21RROD,GSL22DBRO,GSL22RBRA,GSL23DAND,GSL23RNAT,GSL24DTAT,GSL24RKNO,GSL25RTIT,GSL26DSAL,GSL26RKAT,GSL27IGER,GSL27RBIN,GSL28DSCH,GSL28RMAG,GSL29DJOH,GSL29RMOS,GSL30DYAC,GSL30RZIM,GSL31DBRO,GSL31RSCH,GSL32DJAR,GSL32RAUG,GSL33DVRU,GSL33RJOH,GSL34DDAN,GSL34RSWE,GSL35IFRE,GSL35RCAL,GSL36DMUR,GSL36RMUR,GSL37DMCC,GSL37RPEN,GSL38RDIT,GSL39RBOR,GSL39RRYD,GSL40IFRI,GSL40RPET,GSL41RDAL,GSL42DVAL,GSL42RPLU,GSL43DJAC,GSL43RVOE,GSL44DCON,GSL44RZIM,GSL45DAND,GSL45RKLE,GSL46DRAT,GSL46RMCK,GSL47DAND,GSL47RNEW,GSL48DBAL,GSL49DPAR,GSL49RTRA,GSL50DLEU,GSL50RKUR,GSL51DSPI,GSL51RNOV,GSL52DLAV,GSL52RO'C,GSL53RSCH,GSL54DPAL,GSL54RHER,GSL55DHOL,GSL55RGUS,GSL56DHAY,GSL56RMUR,GSL57DSNO,GSL57RFOX,GSL58DRZE,GSL58RGUN,GSL59RBOD,GSL60DLAR,GSL60RBRO,GSL61DWIN,GSL61RNED,GSL62DHAM,GSL62RWIT,GSL63DJAC,GSL63RSTE,GSL63RVOS,GSL64DMCG,GSL64RHIB,GSL65DOHN,GSL65RPET,GSL66DNEU,GSL66LHUT,GSL67DBEN,GSL67RSUM,GSL68DOTT,GSL68RHUR,GSL69DBOE,GSL69RROZ,GSL70DGOM,GSL70RVAN,GSL71DSHA,GSL71RSOI,GSL72DGRE,GSL72RKRU,GSL73DGAP,GSL73RSAP,GSL74DADA,GSL74RGRE,GSL75RARM,GSL76DHON,GSL77DSTU,GSL78DSUB,GSL78RNEU,GSL79DJOE,GSL79RFUE,GSL80DBAR,GSL80RLUG,GSL81DCON,GSL81RBEN,GSL82DDAV,GSL82IROP,GSL82RWIC,GSL83ISEL,GSL83RRET,GSL84DBIR,GSL84RDON,GSL85DCON,GSL85RSNY,GSL86RSPI,GSL87DRIL,GSL87REDM,GSL88DBEA,GSL88RMAC,GSL89DBEN,GSL89RBEH,GSL90DSHE,GSL90RBEH,GSL91DEME,GSL91RSTA,GSL92DBAM,GSL92RPRO,GSL93DPAG,GSL93RPET,GSL94DDOY,GSL94RHUE,GSL95DBIL,GSL95RWOO,GSL96DSWI,GSL96ROLD,GSL97RALL,GSL98DBAR,GSL98RNEY,GSL99DDAH,GSL99RDUC,GSLOWRI,GSU01DGAG,GSU01RJAC,GSU03DCAR,GSU03RSAN,GSU05DKAT,GSU05RHUT,GSU07DLAR,GSU07RGIL,GSU09DSCH,GSU09RLEM,GSU11DDOE,GSU11RNAS,GSU13RJAG,GSU15DSPR,GSU15RTRO,GSU17DSKO,GSU17RMAR,GSU19DALF,GSU19RCAB,GSU21RWAN,GSU23RJAM,GSU25DWES,GSU25RQUI,GSU27DHES,GSU27RREL,GSU29DLOO,GSU29RTOM,GSU31DSMI,GSU31REST,GSU33RKAP,GSUOWRI,geometry,SU_DIST,SL_DIST


In [14]:
#os.mkdir("wi_gen_22_ru")
clean.to_file("wi_gen_22_ru_DRAFT/wi_gen_22_ru.shp")

## Step 6: Follow-ups to check post-export

In [15]:
read = gp.read_file("./wi_gen_22_ru_DRAFT/wi_gen_22_ru.shp")

In [16]:
read[~read["geometry"].is_valid]

,UNIQUE_ID,COUNTYFP,COUNTYNM,CTV,MCD_NAME,CON_DIST,SLDL_DIST,SLDU_DIST,G22ATGDKAU,G22ATGOWRI,G22ATGRTON,G22GOVDEVE,G22GOVIBEG,G22GOVIHAS,G22GOVOWRI,G22GOVRMIC,G22SOSDFOL,G22SOSIMCF,G22SOSLHAR,G22SOSOWRI,G22SOSRLOU,G22TRECZUE,G22TREDRIC,G22TREOWRI,G22TRERLEI,G22USSDBAR,G22USSIPAU,G22USSOWRI,G22USSRJOH,GCON01DROE,GCON01IBAR,GCON01RSTE,GCON02DPOC,GCON02IALE,GCON02ROLS,GCON03DPFA,GCON03RORD,GCON04DMOO,GCON04IRAY,GCON04RROG,GCON05DSOM,GCON05RFIT,GCON06IPOW,GCON06RGRO,GCON07DAUS,GCON07RTIF,GCON08DHAN,GCON08DHOF,GCON08IBOU,GCON08LVAN,GCON08RGAL,GCONOWRI,GSL01DTHE,GSL01RKIT,GSL02DGAS,GSL02RSOR,GSL03RTUS,GSL04DTEA,GSL04RSTE,GSL05DDEU,GSL05RGOE,GSL06DSWI,GSL06RNEU,GSL06RSCH,GSL07DRIE,GSL07RMAR,GSL08DORT,GSL09DCAB,GSL09RANT,GSL10DMAD,GSL11DDRA,GSL12DMYE,GSL12RCAN,GSL13DHAR,GSL13RMIC,GSL14DVIN,GSL14RTUR,GSL15RMAX,GSL16DHAY,GSL17DOMO,GSL17REIS,GSL18DGOY,GSL19DCLA,GSL20DSIN,GSL20RHER,GSL21DJUR,GSL21RROD,GSL22DBRO,GSL22RBRA,GSL23DAND,GSL23RNAT,GSL24DTAT,GSL24RKNO,GSL25RTIT,GSL26DSAL,GSL26RKAT,GSL27IGER,GSL27RBIN,GSL28DSCH,GSL28RMAG,GSL29DJOH,GSL29RMOS,GSL30DYAC,GSL30RZIM,GSL31DBRO,GSL31RSCH,GSL32DJAR,GSL32RAUG,GSL33DVRU,GSL33RJOH,GSL34DDAN,GSL34RSWE,GSL35IFRE,GSL35RCAL,GSL36DMUR,GSL36RMUR,GSL37DMCC,GSL37RPEN,GSL38RDIT,GSL39RBOR,GSL39RRYD,GSL40IFRI,GSL40RPET,GSL41RDAL,GSL42DVAL,GSL42RPLU,GSL43DJAC,GSL43RVOE,GSL44DCON,GSL44RZIM,GSL45DAND,GSL45RKLE,GSL46DRAT,GSL46RMCK,GSL47DAND,GSL47RNEW,GSL48DBAL,GSL49DPAR,GSL49RTRA,GSL50DLEU,GSL50RKUR,GSL51DSPI,GSL51RNOV,GSL52DLAV,GSL52RO'C,GSL53RSCH,GSL54DPAL,GSL54RHER,GSL55DHOL,GSL55RGUS,GSL56DHAY,GSL56RMUR,GSL57DSNO,GSL57RFOX,GSL58DRZE,GSL58RGUN,GSL59RBOD,GSL60DLAR,GSL60RBRO,GSL61DWIN,GSL61RNED,GSL62DHAM,GSL62RWIT,GSL63DJAC,GSL63RSTE,GSL63RVOS,GSL64DMCG,GSL64RHIB,GSL65DOHN,GSL65RPET,GSL66DNEU,GSL66LHUT,GSL67DBEN,GSL67RSUM,GSL68DOTT,GSL68RHUR,GSL69DBOE,GSL69RROZ,GSL70DGOM,GSL70RVAN,GSL71DSHA,GSL71RSOI,GSL72DGRE,GSL72RKRU,GSL73DGAP,GSL73RSAP,GSL74DADA,GSL74RGRE,GSL75RARM,GSL76DHON,GSL77DSTU,GSL78DSUB,GSL78RNEU,GSL79DJOE,GSL79RFUE,GSL80DBAR,GSL80RLUG,GSL81DCON,GSL81RBEN,GSL82DDAV,GSL82IROP,GSL82RWIC,GSL83ISEL,GSL83RRET,GSL84DBIR,GSL84RDON,GSL85DCON,GSL85RSNY,GSL86RSPI,GSL87DRIL,GSL87REDM,GSL88DBEA,GSL88RMAC,GSL89DBEN,GSL89RBEH,GSL90DSHE,GSL90RBEH,GSL91DEME,GSL91RSTA,GSL92DBAM,GSL92RPRO,GSL93DPAG,GSL93RPET,GSL94DDOY,GSL94RHUE,GSL95DBIL,GSL95RWOO,GSL96DSWI,GSL96ROLD,GSL97RALL,GSL98DBAR,GSL98RNEY,GSL99DDAH,GSL99RDUC,GSLOWRI,GSU01DGAG,GSU01RJAC,GSU03DCAR,GSU03RSAN,GSU05DKAT,GSU05RHUT,GSU07DLAR,GSU07RGIL,GSU09DSCH,GSU09RLEM,GSU11DDOE,GSU11RNAS,GSU13RJAG,GSU15DSPR,GSU15RTRO,GSU17DSKO,GSU17RMAR,GSU19DALF,GSU19RCAB,GSU21RWAN,GSU23RJAM,GSU25DWES,GSU25RQUI,GSU27DHES,GSU27RREL,GSU29DLOO,GSU29RTOM,GSU31DSMI,GSU31REST,GSU33RKAP,GSUOWRI,SU_DIST,SL_DIST,geometry
53,BARRON;TOWN OF LAKELAND;1-2,005,BARRON,T,LAKELAND,7,75.0,25.0,183.0,1.0,325.0,187.0,8.0,0.0,1.0,312.0,176.0,7.0,6.0,1.0,318.0,10.0,173.0,1.0,322.0,169.0,0.0,1.0,333.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,169.0,336.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,399.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,157.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,None,None,"MULTIPOLYGON (((-92.00125 45.63970, -92.00047 ..."
171,BROWN;TOWN OF LEDGEVIEW;1-11,009,BROWN,T,LEDGEVIEW,8,88.0,0.0,1941.0,1.0,2513.0,1989.0,36.0

In [17]:
read[~read["geometry"].is_valid].shape[0]

23

Because of geopandas/shapely version on machine exporting and SN will add buffer(0) to validate invalid geometries

## Step 7: Make readme

In [18]:
column_dict.update(script_prod_copy_col_dict)

In [19]:
fields_dict = dict(zip(column_dict.values(), column_dict.keys()))
myKeys = list(fields_dict.keys())
myKeys.sort()
fields_dict = {i: fields_dict[i] for i in myKeys}


### Create README

fields_dict['UNIQUE_ID'] ='Unique ID for each reporting unit based on ward numbers, county name, MCD_NAME, and CTV'
fields_dict['COUNTYFP'] ='3-digit county FIP identifier'
fields_dict['COUNTYNM'] ='County Name '
fields_dict['CTV'] ='Classification as city, town, township, village, etc'
fields_dict['MCD_NAME'] ='Minor Civil Division name'
fields_dict['WARD_FIPS'] ='Ward number'
fields_dict['CON_DIST'] ='2022 Congressional district'
fields_dict['SL_DIST'] ='2022 State Assembly district'
fields_dict['SU_DIST'] ='2022 State Senate district'
fields_dict['GCONOWRI'] ='Congressional scattering/write-in candidates '
fields_dict['GSLOWRI'] ='State Assembly scattering/write-in candidates '
fields_dict['GSUOWRI'] ='State Senate scattering/write-in candidates '

                                          


title = "Wisconsin 2022 General Election Reporting Unit Level Boundaries and Results"
retrieval_date = "11/07/2023"
fields_dict = fields_dict
github_link = "https://github.com/nonpartisan-redistricting-datahub/pber_collection"
file_folder = "./"
source = '''The RDH retrieved 2022 general election reporting unit level results from the Wisconsin Election Commission (https://elections.wi.gov/elections/election-results) 
Reporting units were made by the RDH using 2022 wards retrieved from the Wisconsin Election Commission upon request. Please see "Additional Notes" below for more information '''



def full_readme_text(title, retrieval_date, source, fields_dict, github_link):

#First section of README
    readme_p1 = '''{title}\n
## RDH Date Retrieval
{retrieval_date}

## Sources
{source}

## Fields:
'''.format(title = title, source = source, retrieval_date = retrieval_date)

#Second section of README
    fields_table = pd.DataFrame.from_dict(sorted(fields_dict.items()))
    fields_table.columns = ["Field Name", "Description"]
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False, justify = "left")

#Third section of README
    readme_p3 = '''\n
## Processing Steps
Visit the RDH GitHub and the processing script for this code [here]({github_link})

Please direct questions related to processing this dataset to info@redistrictingdatahub.org.
'''.format(github_link=github_link)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme
if not os.path.exists(file_folder):
    os.mkdir(file_folder)

with open(file_folder+"README_copy.txt", 'w') as tf:
        tf.write(full_readme_text(title, retrieval_date, source, fields_dict, github_link))
